# Database Partitioning: Horizontal vs Vertical

**Partitioning** is a database design technique that divides large tables into smaller, more manageable pieces while maintaining the logical unity of the data. This improves query performance, simplifies maintenance, and enables better resource utilization.

---

## Overview

| Aspect | Horizontal Partitioning | Vertical Partitioning |
|--------|------------------------|----------------------|
| **Division** | By rows | By columns |
| **Also Known As** | Sharding (when distributed) | Normalization / Column splitting |
| **Use Case** | Large row counts | Wide tables with many columns |
| **Data Distribution** | Complete rows in each partition | Subset of columns in each partition |
| **Primary Key** | Duplicated across partitions | Used to join partitions |

## 1. Horizontal Partitioning (Row-Based)

Horizontal partitioning divides a table into multiple tables, each containing a **subset of rows** but all columns.

```
Original Table (Orders):
┌────────┬────────────┬──────────┬─────────┐
│ id     │ date       │ customer │ amount  │
├────────┼────────────┼──────────┼─────────┤
│ 1      │ 2024-01    │ Alice    │ 100     │
│ 2      │ 2024-02    │ Bob      │ 200     │
│ 3      │ 2025-01    │ Carol    │ 150     │
│ 4      │ 2025-02    │ Dave     │ 300     │
└────────┴────────────┴──────────┴─────────┘

After Horizontal Partitioning (by year):

Partition 2024:                    Partition 2025:
┌────────┬────────────┬──────────┬─────────┐   ┌────────┬────────────┬──────────┬─────────┐
│ id     │ date       │ customer │ amount  │   │ id     │ date       │ customer │ amount  │
├────────┼────────────┼──────────┼─────────┤   ├────────┼────────────┼──────────┼─────────┤
│ 1      │ 2024-01    │ Alice    │ 100     │   │ 3      │ 2025-01    │ Carol    │ 150     │
│ 2      │ 2024-02    │ Bob      │ 200     │   │ 4      │ 2025-02    │ Dave     │ 300     │
└────────┴────────────┴──────────┴─────────┘   └────────┴────────────┴──────────┴─────────┘
```

### Benefits
- **Improved query performance**: Queries only scan relevant partitions
- **Easier maintenance**: Archive or drop old partitions without affecting others
- **Parallel processing**: Different partitions can be processed concurrently
- **Better index performance**: Smaller indexes per partition

## 2. Vertical Partitioning (Column-Based)

Vertical partitioning divides a table by **columns**, creating multiple tables with fewer columns each. All partitions share the primary key.

```
Original Table (Users):
┌────────┬──────────┬───────────────────┬─────────────┬──────────────────────┐
│ id     │ name     │ email             │ profile_pic │ bio (large text)     │
├────────┼──────────┼───────────────────┼─────────────┼──────────────────────┤
│ 1      │ Alice    │ alice@email.com   │ [BLOB]      │ Long biography...    │
│ 2      │ Bob      │ bob@email.com     │ [BLOB]      │ Another long bio...  │
└────────┴──────────┴───────────────────┴─────────────┴──────────────────────┘

After Vertical Partitioning:

users_core (hot data):             users_profile (cold data):
┌────────┬──────────┬───────────────────┐   ┌────────┬─────────────┬──────────────────────┐
│ id     │ name     │ email             │   │ id     │ profile_pic │ bio                  │
├────────┼──────────┼───────────────────┤   ├────────┼─────────────┼──────────────────────┤
│ 1      │ Alice    │ alice@email.com   │   │ 1      │ [BLOB]      │ Long biography...    │
│ 2      │ Bob      │ bob@email.com     │   │ 2      │ [BLOB]      │ Another long bio...  │
└────────┴──────────┴───────────────────┘   └────────┴─────────────┴──────────────────────┘
```

### Benefits
- **Reduced I/O**: Only fetch columns you need
- **Better cache utilization**: Hot columns stay in memory
- **Improved row density**: More rows fit per page
- **Security**: Sensitive columns can be isolated

---

## 3. Table Partitioning Strategies

### 3.1 Range Partitioning
Rows are distributed based on a **range of values** in the partition key.

**Best for**: Time-series data, sequential IDs, date-based queries

In [ ]:
from datetime import datetime, date
from typing import Dict, List, Any
from dataclasses import dataclass

@dataclass
class Order:
    id: int
    order_date: date
    customer: str
    amount: float

class RangePartitioner:
    """Range-based partitioning by year."""
    
    def __init__(self):
        self.partitions: Dict[int, List[Order]] = {}
    
    def get_partition_key(self, order: Order) -> int:
        """Determine partition based on order year."""
        return order.order_date.year
    
    def insert(self, order: Order):
        partition_key = self.get_partition_key(order)
        if partition_key not in self.partitions:
            self.partitions[partition_key] = []
        self.partitions[partition_key].append(order)
        print(f"Inserted order {order.id} into partition_{partition_key}")
    
    def query_by_year(self, year: int) -> List[Order]:
        """Partition pruning - only scan relevant partition."""
        return self.partitions.get(year, [])
    
    def show_partitions(self):
        for year, orders in sorted(self.partitions.items()):
            print(f"\nPartition {year}: {len(orders)} orders")
            for o in orders:
                print(f"  - Order {o.id}: {o.customer}, ${o.amount}")

# Example usage
partitioner = RangePartitioner()

orders = [
    Order(1, date(2023, 3, 15), "Alice", 150.00),
    Order(2, date(2023, 7, 20), "Bob", 200.00),
    Order(3, date(2024, 1, 10), "Carol", 175.00),
    Order(4, date(2024, 6, 5), "Dave", 300.00),
    Order(5, date(2025, 2, 1), "Eve", 250.00),
]

for order in orders:
    partitioner.insert(order)

partitioner.show_partitions()

### 3.2 List Partitioning
Rows are distributed based on a **discrete list of values**.

**Best for**: Geographic regions, categories, status codes

In [ ]:
@dataclass
class Customer:
    id: int
    name: str
    region: str
    revenue: float

class ListPartitioner:
    """List-based partitioning by region."""
    
    # Define partition mapping
    REGION_PARTITIONS = {
        'americas': ['US', 'CA', 'MX', 'BR'],
        'europe': ['UK', 'DE', 'FR', 'ES'],
        'asia_pacific': ['JP', 'CN', 'AU', 'IN'],
    }
    
    def __init__(self):
        self.partitions: Dict[str, List[Customer]] = {
            name: [] for name in self.REGION_PARTITIONS
        }
        self.partitions['other'] = []  # Default partition
    
    def get_partition_key(self, region: str) -> str:
        for partition_name, regions in self.REGION_PARTITIONS.items():
            if region in regions:
                return partition_name
        return 'other'
    
    def insert(self, customer: Customer):
        partition_key = self.get_partition_key(customer.region)
        self.partitions[partition_key].append(customer)
        print(f"Customer {customer.name} ({customer.region}) -> partition_{partition_key}")
    
    def query_by_partition(self, partition_name: str) -> List[Customer]:
        return self.partitions.get(partition_name, [])

# Example usage
list_partitioner = ListPartitioner()

customers = [
    Customer(1, "Acme Corp", "US", 50000),
    Customer(2, "TechGmbH", "DE", 75000),
    Customer(3, "Tokyo Ltd", "JP", 60000),
    Customer(4, "Maple Inc", "CA", 45000),
    Customer(5, "OutbackCo", "AU", 55000),
]

for customer in customers:
    list_partitioner.insert(customer)

print("\n--- Query Americas partition ---")
for c in list_partitioner.query_by_partition('americas'):
    print(f"  {c.name}: ${c.revenue:,.0f}")

### 3.3 Hash Partitioning
Rows are distributed using a **hash function** on the partition key.

**Best for**: Even data distribution, avoiding hotspots, no natural range/list

In [ ]:
import hashlib

@dataclass
class User:
    user_id: int
    username: str
    email: str

class HashPartitioner:
    """Hash-based partitioning for even distribution."""
    
    def __init__(self, num_partitions: int = 4):
        self.num_partitions = num_partitions
        self.partitions: Dict[int, List[User]] = {
            i: [] for i in range(num_partitions)
        }
    
    def get_partition_key(self, user_id: int) -> int:
        """Consistent hashing to determine partition."""
        hash_value = hashlib.md5(str(user_id).encode()).hexdigest()
        return int(hash_value, 16) % self.num_partitions
    
    def insert(self, user: User):
        partition_key = self.get_partition_key(user.user_id)
        self.partitions[partition_key].append(user)
    
    def find_user(self, user_id: int) -> User | None:
        """Efficient lookup - only check one partition."""
        partition_key = self.get_partition_key(user_id)
        for user in self.partitions[partition_key]:
            if user.user_id == user_id:
                return user
        return None
    
    def show_distribution(self):
        print("Hash Partition Distribution:")
        for i, users in self.partitions.items():
            print(f"  Partition {i}: {len(users)} users")

# Example: Insert many users to show distribution
hash_partitioner = HashPartitioner(num_partitions=4)

for i in range(1, 101):  # 100 users
    user = User(i, f"user_{i}", f"user{i}@example.com")
    hash_partitioner.insert(user)

hash_partitioner.show_distribution()

# Lookup efficiency
print(f"\nLooking up user 42...")
found = hash_partitioner.find_user(42)
if found:
    print(f"  Found: {found.username} in partition {hash_partitioner.get_partition_key(42)}")

### 3.4 Composite (Multi-Level) Partitioning
Combines multiple strategies, e.g., **range-hash** or **list-range**.

**Best for**: Complex queries, multi-dimensional access patterns

In [ ]:
@dataclass
class Transaction:
    id: int
    transaction_date: date
    region: str
    amount: float

class CompositePartitioner:
    """Composite partitioning: Range (by year) + List (by region)."""
    
    REGIONS = ['NA', 'EU', 'APAC']
    
    def __init__(self):
        # Two-level partition structure: year -> region -> transactions
        self.partitions: Dict[int, Dict[str, List[Transaction]]] = {}
    
    def get_partition_keys(self, txn: Transaction) -> tuple:
        year = txn.transaction_date.year
        region = txn.region if txn.region in self.REGIONS else 'OTHER'
        return (year, region)
    
    def insert(self, txn: Transaction):
        year, region = self.get_partition_keys(txn)
        
        if year not in self.partitions:
            self.partitions[year] = {r: [] for r in self.REGIONS + ['OTHER']}
        
        self.partitions[year][region].append(txn)
    
    def query(self, year: int = None, region: str = None) -> List[Transaction]:
        """Query with partition pruning at both levels."""
        results = []
        
        years_to_scan = [year] if year else self.partitions.keys()
        
        for y in years_to_scan:
            if y not in self.partitions:
                continue
            regions_to_scan = [region] if region else self.partitions[y].keys()
            for r in regions_to_scan:
                if r in self.partitions[y]:
                    results.extend(self.partitions[y][r])
        
        return results
    
    def show_structure(self):
        print("Composite Partition Structure:")
        for year in sorted(self.partitions.keys()):
            print(f"\n  Year {year}:")
            for region, txns in self.partitions[year].items():
                if txns:
                    total = sum(t.amount for t in txns)
                    print(f"    {region}: {len(txns)} txns, ${total:,.2f}")

# Example usage
composite = CompositePartitioner()

transactions = [
    Transaction(1, date(2024, 3, 15), "NA", 1500),
    Transaction(2, date(2024, 7, 20), "EU", 2000),
    Transaction(3, date(2024, 9, 10), "APAC", 1750),
    Transaction(4, date(2025, 1, 5), "NA", 3000),
    Transaction(5, date(2025, 2, 1), "EU", 2500),
    Transaction(6, date(2025, 4, 12), "NA", 1800),
]

for txn in transactions:
    composite.insert(txn)

composite.show_structure()

print("\n--- Query: 2025, NA region ---")
for t in composite.query(year=2025, region='NA'):
    print(f"  Txn {t.id}: ${t.amount:,.2f}")

---

## 4. Vertical Partitioning Example

Splitting a wide table into hot (frequently accessed) and cold (rarely accessed) partitions.

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

# Original wide table
@dataclass
class ProductFull:
    id: int
    name: str
    price: float
    stock: int
    # Cold data (rarely accessed)
    description: str = ""
    specifications: str = ""
    reviews_json: str = ""
    image_urls: str = ""

# Vertically partitioned tables
@dataclass
class ProductCore:
    """Hot partition - frequently accessed columns."""
    id: int
    name: str
    price: float
    stock: int

@dataclass
class ProductDetails:
    """Cold partition - rarely accessed, large columns."""
    id: int  # Foreign key to join
    description: str
    specifications: str
    reviews_json: str
    image_urls: str

class VerticallyPartitionedProducts:
    """Simulates vertical partitioning with separate storage."""
    
    def __init__(self):
        self.core_partition: Dict[int, ProductCore] = {}
        self.details_partition: Dict[int, ProductDetails] = {}
    
    def insert(self, product: ProductFull):
        # Split into partitions
        self.core_partition[product.id] = ProductCore(
            product.id, product.name, product.price, product.stock
        )
        self.details_partition[product.id] = ProductDetails(
            product.id, product.description, product.specifications,
            product.reviews_json, product.image_urls
        )
    
    def get_core(self, product_id: int) -> Optional[ProductCore]:
        """Fast lookup - only accesses hot partition."""
        return self.core_partition.get(product_id)
    
    def get_full(self, product_id: int) -> Optional[ProductFull]:
        """Join both partitions when full data needed."""
        core = self.core_partition.get(product_id)
        details = self.details_partition.get(product_id)
        
        if core and details:
            return ProductFull(
                core.id, core.name, core.price, core.stock,
                details.description, details.specifications,
                details.reviews_json, details.image_urls
            )
        return None

# Example usage
store = VerticallyPartitionedProducts()

product = ProductFull(
    id=1,
    name="Laptop Pro",
    price=1299.99,
    stock=50,
    description="A very long product description..." * 10,
    specifications='{"cpu": "i7", "ram": "16GB", "storage": "512GB SSD"}',
    reviews_json='[{"rating": 5, "text": "Great laptop!"}]',
    image_urls='["img1.jpg", "img2.jpg", "img3.jpg"]'
)

store.insert(product)

# Most queries only need core data (fast)
print("Quick lookup (core partition only):")
core = store.get_core(1)
print(f"  {core.name}: ${core.price}, Stock: {core.stock}")

# Product detail page needs full data
print("\nFull lookup (join both partitions):")
full = store.get_full(1)
print(f"  Description length: {len(full.description)} chars")

---

## 5. Performance Benefits & Use Cases

### Performance Benefits

| Benefit | Horizontal | Vertical |
|---------|------------|----------|
| **Partition Pruning** | ✅ Skip irrelevant row partitions | ✅ Skip unused column partitions |
| **Parallel Query** | ✅ Scan partitions concurrently | ✅ Join partitions in parallel |
| **Index Size** | ✅ Smaller indexes per partition | ✅ More relevant indexes |
| **Cache Efficiency** | Moderate | ✅ Hot columns stay cached |
| **Maintenance** | ✅ Easy archival of old partitions | Moderate complexity |

### Common Use Cases

**Horizontal Partitioning**
- Time-series data (logs, events, transactions)
- Multi-tenant SaaS applications
- Geographic data distribution
- Archiving old data

**Vertical Partitioning**
- Tables with LOB (Large Object) columns
- Separating PII for security compliance
- Hot/cold data separation
- Microservices data isolation

In [ ]:
# Demonstrating partition pruning benefit
import time

class PartitionPruningDemo:
    """Compare full scan vs partition-aware query."""
    
    def __init__(self):
        # Simulate 1M orders across 5 years
        self.all_orders = []
        self.partitioned_orders: Dict[int, List[dict]] = {}
        
        for year in range(2020, 2026):
            self.partitioned_orders[year] = []
        
        # Generate sample data
        for i in range(100000):  # 100K orders for demo
            year = 2020 + (i % 6)
            order = {'id': i, 'year': year, 'amount': (i % 500) + 50}
            self.all_orders.append(order)
            self.partitioned_orders[year].append(order)
    
    def full_table_scan(self, target_year: int) -> list:
        """Scan all rows - no partitioning."""
        return [o for o in self.all_orders if o['year'] == target_year]
    
    def partition_scan(self, target_year: int) -> list:
        """Only scan relevant partition."""
        return self.partitioned_orders.get(target_year, [])

demo = PartitionPruningDemo()

# Benchmark full scan
start = time.perf_counter()
result1 = demo.full_table_scan(2024)
full_scan_time = time.perf_counter() - start

# Benchmark partition scan
start = time.perf_counter()
result2 = demo.partition_scan(2024)
partition_time = time.perf_counter() - start

print(f"Full Table Scan: {len(result1)} rows in {full_scan_time*1000:.2f}ms")
print(f"Partition Scan:  {len(result2)} rows in {partition_time*1000:.2f}ms")
print(f"\nSpeedup: {full_scan_time/partition_time:.1f}x faster with partitioning")

---

## 6. SQL Examples (PostgreSQL)

```sql
-- Range Partitioning Example
CREATE TABLE orders (
    id SERIAL,
    order_date DATE NOT NULL,
    customer_id INT,
    amount DECIMAL(10,2)
) PARTITION BY RANGE (order_date);

CREATE TABLE orders_2024 PARTITION OF orders
    FOR VALUES FROM ('2024-01-01') TO ('2025-01-01');

CREATE TABLE orders_2025 PARTITION OF orders
    FOR VALUES FROM ('2025-01-01') TO ('2026-01-01');

-- List Partitioning Example
CREATE TABLE customers (
    id SERIAL,
    name TEXT,
    region TEXT NOT NULL
) PARTITION BY LIST (region);

CREATE TABLE customers_americas PARTITION OF customers
    FOR VALUES IN ('US', 'CA', 'MX', 'BR');

CREATE TABLE customers_europe PARTITION OF customers
    FOR VALUES IN ('UK', 'DE', 'FR', 'ES');

-- Hash Partitioning Example
CREATE TABLE users (
    user_id INT NOT NULL,
    username TEXT,
    email TEXT
) PARTITION BY HASH (user_id);

CREATE TABLE users_p0 PARTITION OF users
    FOR VALUES WITH (MODULUS 4, REMAINDER 0);
CREATE TABLE users_p1 PARTITION OF users
    FOR VALUES WITH (MODULUS 4, REMAINDER 1);
-- ... etc for p2, p3
```

---

## 📌 Key Takeaways

1. **Horizontal Partitioning** divides tables by **rows** - ideal for large datasets with time-based or categorical access patterns

2. **Vertical Partitioning** divides tables by **columns** - ideal for wide tables with mixed hot/cold data

3. **Partitioning Strategies**:
   - **Range**: Best for sequential/time-based data
   - **List**: Best for discrete categories (regions, types)
   - **Hash**: Best for even distribution, avoiding hotspots
   - **Composite**: Combines strategies for complex access patterns

4. **Performance gains** come from:
   - Partition pruning (skip irrelevant partitions)
   - Parallel processing across partitions
   - Smaller indexes and better cache utilization

5. **Choose partitioning based on**:
   - Query patterns (how data is accessed)
   - Data growth patterns
   - Maintenance requirements (archiving, purging)
   - Performance bottlenecks